In [1]:
import pathlib
import json
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
# !pip install folium
import folium

from pathlib import Path

from collections import Counter

## Analyse data

In [2]:
df_sum = None
for df_path in tqdm(list(Path("../data/local/tables/").iterdir())[-10:]):
    if df_sum is None: 
        df_sum = pd.read_csv(df_path, encoding='utf-8', low_memory=False)
    else : 
        df_sum.append(pd.read_csv(df_path, encoding='utf-8', low_memory=False))

100%|██████████| 10/10 [00:10<00:00,  1.03s/it]


In [3]:
df_sum[df_sum.duplicated()]

,id,imei,name,stateCode,stateName,lat,lng,speed,orientation,gpstime,routeId,routeName,routeColour,inDepo,busNumber,perevId,perevName,remark,online,idBusTypes


In [4]:
df_sum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280232 entries, 0 to 280231
Data columns (total 20 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           280232 non-null  int64  
 1   imei         280232 non-null  int64  
 2   name         280232 non-null  object 
 3   stateCode    280232 non-null  object 
 4   stateName    280232 non-null  object 
 5   lat          280232 non-null  float64
 6   lng          280232 non-null  float64
 7   speed        280232 non-null  float64
 8   orientation  280232 non-null  float64
 9   gpstime      280232 non-null  object 
 10  routeId      280232 non-null  int64  
 11  routeName    280232 non-null  object 
 12  routeColour  280232 non-null  object 
 13  inDepo       280232 non-null  bool   
 14  busNumber    280232 non-null  object 
 15  perevId      280232 non-null  int64  
 16  perevName    280232 non-null  object 
 17  remark       280232 non-null  object 
 18  online       280232 non-

### SQL Shema bouild

In [7]:
df_sum['inDepo'].value_counts()

False    271876
True       8356
Name: inDepo, dtype: int64

In [61]:
# df_sum[['routeId', 'routeName', 'routeColour']].value_counts()
# df_sum[['imei', 'name', 'busNumber', 'perevId']].value_counts()
df_sum[['idBusTypes']].value_counts()

idBusTypes
1             200704
2              79528
dtype: int64

In [63]:
df_sum[['imei', 'name', 'busNumber', 'perevId', 'idBusTypes']].value_counts()

imei             name  busNumber  perevId  idBusTypes
355227045371655  A1    6513       7        1             5521
355227046638821  A211  1181       13       1             5328
355228042084283  A207  1032       13       1             5239
355227045540192  A91   7762       1        1             4498
355227044854750  A22   7665       1        1             4483
                                                         ... 
355228043199080  H103  8716       12       1               18
355227045539491  Am86  1027       2        1               10
355228043162351  H107  9138       12       1                5
355227045600855  A142  377        6        2                2
355227046447371  H65   355        6        2                1
Length: 98, dtype: int64

In [42]:
a = df_sum[['remark']].value_counts()

In [57]:
df_sum[df_sum['imei'] == 355227045541257][['imei', 'name', 'busNumber', 'remark']].value_counts()


imei             name  busNumber  remark    
355227045541257  A57   9043       0325 DNSNK    2007
                                  9043 DNSNK    1870
dtype: int64

In [53]:
from collections import Counter

In [58]:
# Counter([k[0] for k in a.to_dict()])

In [59]:
# a.to_dict()

0   id           280232 non-null  int64  

### USELESS ?!
 3   stateCode    280232 non-null  object #only value is 'used'
 
 4   stateName    280232 non-null  object #only value is 'used'
 
 18  online       280232 non-null  bool   #only value is 'True'
 

### ROUTES
 10  routeId      280232 non-null  int64  
 
 11  routeName    280232 non-null  object 
 
 12  routeColour  280232 non-null  object 
    

### OWNER
 15  perevId      280232 non-null  int64  
 
 16  perevName    280232 non-null  object 


### WEEHCLE 
 1   imei         280232 non-null  int64  
 
 2   name         280232 non-null  object 
 
 14  busNumber    280232 non-null  object 
 
 17  remark       280232 non-null  object 
 
 19  idBusTypes   280232 non-null  int64  

### GPS
 5   lat          280232 non-null  float64
 
 6   lng          280232 non-null  float64
 
 7   speed        280232 non-null  float64
 
 8   orientation  280232 non-null  float64
 
 9   gpstime      280232 non-null  object 
 
13  inDepo       280232 non-null  bool   


## CODE 
```
Table rowdata.gpsdata {
id int [pk, increment]
lat float
lng float
speed float
orientation float
gpstime datetime
inDepo bool

routeId int [ref: > E.id]

response_datetime datetime

edited datetime

Indexes {
(id) [pk]
}
}

Table rowdata.vehicle as E {
id int [pk, increment]
imei int
name varchar
busNumber int
remark varchar

perevId int [ref: > O.id]
routeId int [ref: > R.id]

edited datetime

Indexes {
(id) [pk]
}
}

Table rowdata.owner as O {
id int [pk, increment]
perevName varchar
edited datetime

Indexes {
(id) [pk]
}
}

Table rowdata.route as R {
id int [pk, increment]
routeName varchar
routeColour varchar

etited datetime

Indexes {
(id) [pk]
}
}
```

In [2]:
import psycopg2 

In [3]:
import os

In [6]:
os.environ['RDS_HOSTNAME'], os.environ['RDS_PTETA_DB_PASSWORD']

('pteta-db-test2.cvulqh76zvum.eu-central-1.rds.amazonaws.com', 'aspjkJ_Bo31')

In [7]:
conn = psycopg2.connect(
    host=os.environ['RDS_HOSTNAME'],
    database="pteta_db",
    user="postgres",
    password=os.environ['RDS_PTETA_DB_PASSWORD'])

In [8]:
type(conn)

psycopg2.extensions.connection

In [9]:
cursor = conn.cursor()

In [10]:
cursor.execute("select * from information_schema.tables")
cursor.fetchall()

[('pteta_db',
  'pteta',
  'gpsdata',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None),
 ('pteta_db',
  'pteta',
  'owner',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None),
 ('pteta_db',
  'pteta',
  'route',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None),
 ('pteta_db',
  'pg_catalog',
  'pg_type',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None),
 ('pteta_db',
  'pg_catalog',
  'pg_foreign_table',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None),
 ('pteta_db',
  'pteta',
  'vehicle',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None),
 ('pteta_db',
  'pg_catalog',
  'pg_roles',
  'VIEW',
  None,
  None,
  None,
  None,
  None,
  'NO',
  'NO',
  None),
 ('pteta_db',
  'pg_catalog',
  'pg_settings',
  'VIEW',
  None,
  None,
  None,
  None,
  None,
  'NO',
  'NO',
  None),
 ('pteta_db',
  'pg_cata

In [56]:
cursor.execute("SELECT * FROM pteta.owner;")
cursor.fetchall()

[]

In [15]:
id, perevName = 6, 'ЧТУ'
with conn.cursor() as cur: 
    sql = f"""SELECT EXISTS(SELECT * FROM pteta.owner where "perevName" = '{perevName}' AND "id" = {id});"""
    cursor.execute(sql)
    print(cursor.fetchall())

[(True,)]


In [17]:
from dataclasses import dataclass

@dataclass(unsafe_hash=True)
class TransportOperator:
    """
    Column name ralations dataclass | DB | HTTP request
    id: int   | id        | perevId
    name: str | perevName | perevName
    """
    id: int
    name: str

In [84]:
def isTransportOperatorInTable(
    connection: psycopg2.extensions.connection, 
    trans_operator: TransportOperator
): 
    with connection.cursor() as cur: 
        sql = f"""SELECT EXISTS(SELECT * FROM pteta.owner """ +\
            f"""WHERE "id" = {trans_operator.id} AND "perevName" = '{trans_operator.name}' );"""
        try:
            cursor.execute(sql)
            return cursor.fetchone()[0]
        except psycopg2.errors.InFailedSqlTransaction as err:
            print(err)
            connection.rollback()
            raise err
            
    
def insertTransportOperatorInTable(
    connection: psycopg2.extensions.connection, 
    trans_operator: TransportOperator
): 
    with connection.cursor() as cur: 
        sql = f"""INSERT INTO pteta.owner("id", "perevName")""" +\
            f"""VALUES ({trans_operator.id}, '{trans_operator.name}');"""
        cursor.execute(sql)
        connection.commit()
        
        
def getTransportOperatorTable(
    connection: psycopg2.extensions.connection
):
    with connection.cursor() as cur: 
        sql = f"""SELECT ID, "perevName" FROM pteta.owner;"""
        try:
            cursor.execute(sql)
            return [TransportOperator(rec[0], rec[1]) for rec in cursor.fetchall()]
        except psycopg2.errors.InFailedSqlTransaction as err:
            print(err)
            connection.rollback()
            raise err

In [85]:
getTransportOperatorTable(conn)

[TransportOperator(id=6, name='ЧТУ'),
 TransportOperator(id=12, name='Vitaliy2019'),
 TransportOperator(id=7, name='Віталій / Олег Бойко '),
 TransportOperator(id=1, name='Денисівка'),
 TransportOperator(id=13, name='Олег Валентинович 39 марш'),
 TransportOperator(id=2, name='Тарас')]

In [ ]:
for perevId, perevName in  df_sum[["perevId", "perevName"]].drop_duplicates().values:
    trans_operator = TransportOperator(perevId, perevName)
    
    if (not isTransportOperatorInTable(conn, trans_operator)): 
        insertTransportOperatorInTable(conn, trans_operator)

In [96]:
df_sum[["perevId", "perevName"]].drop_duplicates().values

array([[6, 'ЧТУ'],
       [12, 'Vitaliy2019'],
       [7, 'Віталій / Олег Бойко '],
       [1, 'Денисівка'],
       [13, 'Олег Валентинович 39 марш'],
       [2, 'Тарас']], dtype=object)

### ROUTES

10 routeId 280232 non-null int64

11 routeName 280232 non-null object

12 routeColour

In [104]:
@dataclass(unsafe_hash=True)
class TransportRoute:
    """
    Column name ralations 
    dataclass   | DB          | HTTP request
    ------------|-------------|------------
    id: int     | id          | routeId
    name: str   | routeName   | routeName
    colour: str | routeColour | routeColour
    """
    id: int
    name: str
    colour: str

In [94]:
from typing import List

In [105]:
def isTransportRouteInTable(
    connection: psycopg2.extensions.connection, 
    trans_route: TransportRoute
)-> bool : 
    with connection.cursor() as cur: 
        sql = f"""SELECT EXISTS(SELECT * FROM pteta.route """ +\
            f"""WHERE "id" = {trans_route.id} AND "routeName" = '{trans_route.name}' );"""
        try:
            cursor.execute(sql)
            return cursor.fetchone()[0]
        except psycopg2.errors.InFailedSqlTransaction as err:
            print(err)
            connection.rollback()
            raise err
            
    
def insertTransportRouteInTable(
    connection: psycopg2.extensions.connection, 
    trans_route: TransportRoute
) -> None:
    with connection.cursor() as cur: 
        sql = f"""INSERT INTO pteta.route("id", "routeName", "routeColour")""" +\
            f"""VALUES ({trans_route.id}, '{trans_route.name}', '{trans_route.colour}');"""
        cursor.execute(sql)
        connection.commit()
        
        
def getTransportOperatorTable(
    connection: psycopg2.extensions.connection
) -> List[TransportRoute]:
    with connection.cursor() as cur: 
        sql = f"""SELECT "id", "routeName", "routeColour" FROM pteta.route;"""
        try:
            cursor.execute(sql)
            return [TransportRoute(rec[0], rec[1], rec[2]) for rec in cursor.fetchall()]
        except psycopg2.errors.InFailedSqlTransaction as err:
            print(err)
            connection.rollback()
            raise err

In [97]:
df_sum[["routeId", "routeName", "routeColour"]].drop_duplicates().values

array([[37, 'T', 'coral'],
       [31, '6/6a', 'deeppink'],
       [21, '38', 'deeppink'],
       [41, '10A', 'black'],
       [20, 'A', 'navy'],
       [42, '39', 'coral'],
       [23, '19', 'teal'],
       [19, '10', 'black'],
       [11, '9', 'magenta'],
       [2, '11', 'green'],
       [6, '4', 'magenta'],
       [27, '3/3a', 'green'],
       [16, '2', 'green'],
       [4, '5', 'orange'],
       [3, '12', 'blue'],
       [12, '20', 'maroon'],
       [7, '6', 'sienna'],
       [10, '34', 'navy'],
       [45, '1', 'navy'],
       [9, '27', 'black'],
       [38, '1T', 'grey'],
       [49, '2A', 'red'],
       [5, '13', 'brown'],
       [33, '8T', 'coral'],
       [17, '23', 'red'],
       [48, '1A', 'olive'],
       [14, '9A', 'magenta'],
       [26, '4T', 'purple'],
       [25, '2T', 'olive'],
       [39, '5T', 'maroon'],
       [34, '11/3', 'teal']], dtype=object)

In [99]:
isTransportRouteInTable(conn, TransportRoute(1, 'a', 'routeColour'))

False

In [106]:
for routeId, routeName, routeColour in df_sum[["routeId", "routeName", "routeColour"]].drop_duplicates().values:
    trans_operator = TransportRoute(routeId, routeName, routeColour)
    
    if (not isTransportRouteInTable(conn, trans_operator)): 
        insertTransportRouteInTable(conn, trans_operator)
        
getTransportOperatorTable(conn)

[TransportRoute(id=37, name='T', colour='coral'),
 TransportRoute(id=31, name='6/6a', colour='deeppink'),
 TransportRoute(id=21, name='38', colour='deeppink'),
 TransportRoute(id=41, name='10A', colour='black'),
 TransportRoute(id=20, name='A', colour='navy'),
 TransportRoute(id=42, name='39', colour='coral'),
 TransportRoute(id=23, name='19', colour='teal'),
 TransportRoute(id=19, name='10', colour='black'),
 TransportRoute(id=11, name='9', colour='magenta'),
 TransportRoute(id=2, name='11', colour='green'),
 TransportRoute(id=6, name='4', colour='magenta'),
 TransportRoute(id=27, name='3/3a', colour='green'),
 TransportRoute(id=16, name='2', colour='green'),
 TransportRoute(id=4, name='5', colour='orange'),
 TransportRoute(id=3, name='12', colour='blue'),
 TransportRoute(id=12, name='20', colour='maroon'),
 TransportRoute(id=7, name='6', colour='sienna'),
 TransportRoute(id=10, name='34', colour='navy'),
 TransportRoute(id=45, name='1', colour='navy'),
 TransportRoute(id=9, name='27'

### WEEHCLE
1 imei 280232 non-null int64

2 name 280232 non-null object

14 busNumber 280232 non-null object

17 remark 280232 non-null object

19 idBusTypes 280232 non-null int64

In [100]:
id
imei
name
busNumber
remark

perevId
routeId

edited

[]

In [134]:
@dataclass(unsafe_hash=True)
class TransportVehicle:
    """
    Column name ralations 
    dataclass   | DB          | HTTP request
    ------------|-------------|------------
    imei: int   | imei        | imei
    name: str   | routeName   | routeName
    colour: str | routeColour | routeColour
    """
    imei: int
    name: str
    busNumber: str
    remark: str
    perevId: int
    routeId: int

In [130]:
# df_sum.head(3)

In [131]:
dt = df_sum[["imei", "name", "busNumber", "remark", "perevId", "routeId"]].drop_duplicates().values
dt.shape

(148, 6)

In [132]:
# df_sum[["imei", "name", "busNumber", "remark", "perevId", "routeId"]].value_counts()

In [129]:
# len(set(dt[:, 3]))

99

In [133]:
# "imei", "name", "busNumber", "remark" (99) ->  "perevId", "routeId" = 148

In [184]:
def isTransportVehicleInTable(
    connection: psycopg2.extensions.connection, 
    trans_vehicle: TransportVehicle
)-> bool : 
    with connection.cursor() as cur: 
        sql = f"""SELECT EXISTS(SELECT * FROM pteta.vehicle """ +\
            f"""WHERE "imei" = '{trans_vehicle.imei}' """ +\
            f"""AND "name" = '{trans_vehicle.name}' """ +\
            f"""AND "busNumber" = '{trans_vehicle.busNumber}' """ +\
            f"""AND "remark" = '{trans_vehicle.remark}' """ +\
            f"""AND "perevId" = {trans_vehicle.perevId} """ +\
            f"""AND "routeId" = {trans_vehicle.routeId});"""
        try:
            cursor.execute(sql)
            return cursor.fetchone()[0]
        except psycopg2.errors.InFailedSqlTransaction as err:
            print(err)
            connection.rollback()
            raise err

        
def insertTransportVehicleInTable(
    connection: psycopg2.extensions.connection, 
    trans_vehicle: TransportVehicle
) -> None:
    with connection.cursor() as cur: 
        sql = f"""INSERT INTO pteta.vehicle("imei", "name", "busNumber", "remark", "perevId", "routeId")""" +\
            f"""VALUES ('{trans_vehicle.imei}', '{trans_vehicle.name}', '{trans_vehicle.busNumber}', """ +\
            f"""'{trans_vehicle.remark}', {trans_vehicle.perevId},  {trans_vehicle.routeId});"""
        
        cursor.execute(sql)
        connection.commit()
        
        
def getTransportVehicleTable(
    connection: psycopg2.extensions.connection
) -> List[TransportVehicle]:
    with connection.cursor() as cur: 
        sql = f"""SELECT "imei", "name", "busNumber", "remark", "perevId", "routeId" FROM pteta.vehicle;"""
        try:
            cursor.execute(sql)
            return [TransportVehicle(rec[0], rec[1], rec[2], rec[3], rec[4], rec[5]) for rec in cursor.fetchall()]
        except psycopg2.errors.InFailedSqlTransaction as err:
            print(err)
            connection.rollback()
            raise err

In [152]:
dt = df_sum[["imei", "name", "busNumber", "remark", "perevId", "routeId"]].drop_duplicates().values
dt[:3, :]

array([[355227045600830, 'A178', '310', 'Тролейбус 310 DNSNK', 6, 37],
       [355227046451662, 'H76', '350', 'Тролейбус 350 DNSNTNK', 6, 31],
       [355227045369527, 'A6', '3557', '3557 DNSNK ', 12, 21]],
      dtype=object)

In [172]:
conn.rollback()

In [179]:
conn

<connection object at 0x000001AA6A5EFE10; dsn: 'user=postgres password=xxx dbname=pteta_db host=pteta-db-test2.cvulqh76zvum.eu-central-1.rds.amazonaws.com', closed: 0>

In [186]:
trans_vehicle = TransportVehicle(355227045369527, 'A6', '3557', '3557 DNSNK ', 12, 21)
isTransportVehicleInTable(conn, trans_vehicle)

False

In [188]:
insertTransportVehicleInTable(conn, trans_vehicle)

In [189]:
getTransportVehicleTable(conn)

[TransportVehicle(imei='355227045369527', name='A6', busNumber=3557, remark='3557 DNSNK ', perevId=12, routeId=21)]

In [192]:
dt = df_sum[["imei", "name", "busNumber", "remark", "perevId", "routeId"]].drop_duplicates().values

for row in tqdm(dt):
    trans_vehicle = TransportVehicle(*row)
    
    if (not isTransportVehicleInTable(conn, trans_vehicle)): 
        insertTransportVehicleInTable(conn, trans_vehicle)
        
getTransportVehicleTable(conn)

100%|██████████| 148/148 [00:25<00:00,  5.88it/s]


[TransportVehicle(imei='355227045600830', name='A178', busNumber='310', remark='Тролейбус 310 DNSNK', perevId=6, routeId=37),
 TransportVehicle(imei='355227046451662', name='H76', busNumber='350', remark='Тролейбус 350 DNSNTNK', perevId=6, routeId=31),
 TransportVehicle(imei='355227045369527', name='A6', busNumber='3557', remark='3557 DNSNK ', perevId=12, routeId=21),
 TransportVehicle(imei='355227045540176', name='A83', busNumber='5150', remark=' 5150 DNS', perevId=7, routeId=41),
 TransportVehicle(imei='355227046453387', name='H75', busNumber='3627', remark='3627 DNSNTNK', perevId=1, routeId=20),
 TransportVehicle(imei='355228042084283', name='A207', busNumber='1032', remark='1032 DNSNK', perevId=13, routeId=42),
 TransportVehicle(imei='355227045371655', name='A1', busNumber='6513', remark='6513 DNSNK', perevId=7, routeId=23),
 TransportVehicle(imei='355227046451407', name='H68', busNumber='0855', remark='0855 DNSNTNK', perevId=13, routeId=42),
 TransportVehicle(imei='355227046578332

In [108]:
trans_operator in trans_routes_set

True

In [10]:
df_sum[["perevId", "perevName"]].value_counts().

perevId  perevName                
1        Денисівка                    90233
6        ЧТУ                          79528
7        Віталій / Олег Бойко         31333
2        Тарас                        28719
12       Vitaliy2019                  27756
13       Олег Валентинович 39 марш    22663
dtype: int64

In [167]:
# sum_df.groupby('imei').size().reset_index(name='count').sort_values(['count'], ascending=False)